In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
from torch.distributions import multivariate_normal as mn

In [2]:
class FeedForward(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(FeedForward, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size,hidden_size)
        self.h2o = nn.Linear(hidden_size,output_size)
        self.sigmoid = nn.LogSigmoid()

    def forward(self,input):
        hidden = self.i2h(input)
        output = self.h2o(hidden)
        output = self.sigmoid(output)
        return output

In [3]:
def generate_data(samp_size):
    mean_1 = torch.Tensor([4,4])
    covar_1 = torch.Tensor([[2,1],[1,2]])
    sampler_1 = mn.MultivariateNormal(mean_1,covar_1)

    mean_2 = torch.Tensor([-4,-4])
    covar_2 = torch.Tensor([[2,1],[1,2]])
    sampler_2 = mn.MultivariateNormal(mean_2,covar_2)

    sample_lebel_1 = sampler_1.sample(sample_shape=(int(samp_size/2),1))
    sample_lebel_2 = sampler_2.sample(sample_shape=(int(samp_size/2),1))

    return sample_lebel_1,sample_lebel_2

In [4]:
sample_lebel_1,sample_lebel_2 = generate_data(10000)

In [5]:
sample_lebel_1.size()

torch.Size([5000, 1, 2])

In [6]:
sample_lebel_2.size()

torch.Size([5000, 1, 2])

In [17]:
def train(feedforward, sample_lebel_1, sample_lebel_2, epochs=1, learning_rate=0.001):
    optimizer = optim.Adam(feedforward.parameters(), lr=learning_rate)
    cnt_samp_1 = 0
    cnt_samp_2 = 0
    iters = 0
    for i in range(epochs):
        while cnt_samp_1<5000 or cnt_samp_2<5000:
            iters+=1
            # selecting one of the classes randomly
            if random.random()>0.5:
                if cnt_samp_1<5000:
                    input_tensor = sample_lebel_1[cnt_samp_1]
                    label_tensor = torch.tensor(1,dtype=torch.float32).view(1,1)
                    cnt_samp_1+=1
                else:
                    input_tensor = sample_lebel_2[cnt_samp_2]
                    label_tensor = torch.tensor(0,dtype=torch.float32).view(1,1)
                    cnt_samp_2+=1
            else:
                if cnt_samp_2<5000:
                    input_tensor = sample_lebel_2[cnt_samp_2]
                    label_tensor = torch.tensor(0,dtype=torch.float32).view(1,1)
                    cnt_samp_2+=1
                else:
                    input_tensor = sample_lebel_1[cnt_samp_1]
                    label_tensor = torch.tensor(1,dtype=torch.float32).view(1,1)
                    cnt_samp_1+=1

            output = feedforward(input_tensor)

            loss = torch.abs(output - label_tensor)

            if iters%100==0:
                print("Loss: ",loss)

            optimizer.zero_grad()

            loss.backward(retain_graph=True)
            optimizer.step()

In [10]:
input_size = 2
hidden_size = 10
output_size = 1

In [11]:
feedforward = FeedForward(2,10,1)

In [18]:
train(feedforward,sample_lebel_1,sample_lebel_2)

Loss:  tensor([[ 0.3336]])
Loss:  tensor([[ 1.3639]])
Loss:  tensor([[ 1.2016]])
Loss:  tensor([[ 0.2472]])
Loss:  tensor([[ 1.1117]])
Loss:  tensor([[ 0.1319]])
Loss:  tensor([[ 1.0715]])
Loss:  tensor([[ 1.0445]])
Loss:  tensor([[ 1.0501]])
Loss:  tensor([[ 1.0311]])
Loss:  tensor([[ 1.0320]])
Loss:  tensor(1.00000e-02 *
       [[ 4.2471]])
Loss:  tensor(1.00000e-02 *
       [[ 1.4336]])
Loss:  tensor(1.00000e-02 *
       [[ 1.2684]])
Loss:  tensor(1.00000e-02 *
       [[ 1.6474]])
Loss:  tensor([[ 1.0070]])
Loss:  tensor(1.00000e-03 *
       [[ 9.2128]])
Loss:  tensor(1.00000e-03 *
       [[ 7.7754]])
Loss:  tensor(1.00000e-03 *
       [[ 8.8178]])
Loss:  tensor([[ 1.0062]])
Loss:  tensor(1.00000e-03 *
       [[ 5.8602]])
Loss:  tensor(1.00000e-03 *
       [[ 4.4844]])
Loss:  tensor(1.00000e-03 *
       [[ 3.9568]])
Loss:  tensor(1.00000e-03 *
       [[ 3.9271]])
Loss:  tensor(1.00000e-03 *
       [[ 4.6786]])
Loss:  tensor([[ 1.0036]])
Loss:  tensor([[ 1.0029]])
Loss:  tensor(1.000

In [21]:
feedforward.i2h.weight

Parameter containing:
tensor([[-0.5739, -0.4829],
        [ 0.3236, -0.5658],
        [-0.6228, -0.3828],
        [ 0.2819,  0.0272],
        [-0.4898,  0.0098],
        [-0.1354,  0.3207],
        [ 0.4265, -0.0620],
        [-0.6739,  0.3346],
        [ 0.4202,  0.3044],
        [-0.5134, -0.4392]])

In [22]:
feedforward.i2h.bias

Parameter containing:
tensor([ 1.2533, -2.3468,  1.3592,  1.1656, -1.7957, -2.6047, -2.4619,
        -2.6159,  1.0329,  0.0351])

In [24]:
feedforward.h2o.weight.size()

torch.Size([1, 10])

In [25]:
feedforward.h2o.bias

Parameter containing:
tensor([ 0.6903])

In [26]:
feedforward.i2h.weight.size()

torch.Size([10, 2])

In [27]:
def evaluate(feedforward,input_tensor):
    output = feedforward(input_tensor)
    return output

In [35]:
input_tensor = sample_lebel_1[0]

In [29]:
evaluate(feedforward,input_tensor)

tensor(1.00000e-05 *
       [[-2.6941]])

In [30]:
input_tensor

tensor([[ 4.8754,  3.7492]])

In [31]:
input_tensor[0] = 0.0

In [34]:
input_tensor

tensor([[ 0.,  0.]])

In [36]:
sample_lebel_1[0]

tensor([[ 0.,  0.]])

In [37]:
input_tensor = torch.tensor([ 4.8754,  3.7492])

In [38]:
input_tensor

tensor([ 4.8754,  3.7492])

In [39]:
sample_lebel_1[0]

tensor([[ 0.,  0.]])

In [42]:
sample_lebel_1[0][0][1] = 3.7492

In [43]:
sample_lebel_1[0]

tensor([[ 0.0000,  3.7492]])

In [44]:
evaluate(feedforward,sample_lebel_1[0])

tensor(1.00000e-05 *
       [[-5.4120]])

In [45]:
sample_lebel_1[0]

tensor([[ 0.0000,  3.7492]])

In [49]:
x = evaluate(feedforward,sample_lebel_1[0])

In [50]:
1/x

tensor([[-18477.6113]])

In [51]:
1/x - 1

tensor([[-18478.6113]])

In [54]:
-torch.log(1/torch.exp(x) -1)

tensor([[ 9.8243]])